# SCRIPT GENERADOR DE JOBS DE INGESTAS
Genera jobs de hammurabis y kirbys
1) Rellenar nombres de hammurabis y kirbys: `hammurabis`, `kirbys`
2) Rellenar version de despliegue de hammurabis y kirbys: `version`
3) Rellenar paquete del repositorio (para configurar correctamente ruta de artifactory): `package`
4) Rellenar `smartcleaner` y `smartcleaner_kind` si aplica

El resultado del script se almacena por país en `.\jobs_ingesta\`

In [1]:
import json
import shutil
import os

In [2]:
smartcleaner = True
smartcleaner_kind = "stg" #TODO, automatizar ruta de raw para cuando sea tipo raw

paises_dict = {
    'europa': '01',
    'america': '02'}

hammurabis = [
    't_kfcm_fdbk_filled_form_field-qlm-kfcm_ext_mvpi.conf',
    't_kfcm_fdbk_saturation_rules-qlm-kfcm_ext_mvpi.conf',
    't_kfcm_feedback_filled_form-qlm-kfcm_ext_mvpi.conf',
    't_kfcm_feedback_form_field-qlm-kfcm_ext_mvpi.conf',
    't_kfcm_feedback_form_shown-qlm-kfcm_ext_mvpi.conf',
    't_kfcm_feedback_forms-qlm-kfcm_ext_mvpi.conf',
    't_kfcm_feedback_options-qlm-kfcm_ext_mvpi.conf',
]
kirbys = [
    't_kfcm_fdbk_filled_form_field-inm-stg_kfcm_ext.conf',
    't_kfcm_fdbk_saturation_rules-inm-stg_kfcm_ext.conf',
    't_kfcm_feedback_filled_form-inm-stg_kfcm_ext.conf',
    't_kfcm_feedback_form_field-inm-stg_kfcm_ext.conf',
    't_kfcm_feedback_form_shown-inm-stg_kfcm_ext.conf',
    't_kfcm_feedback_forms-inm-stg_kfcm_ext.conf',
    't_kfcm_feedback_options-inm-stg_kfcm_ext.conf',
]
version = '2.1.2'#version de confs masterizados
package = "com.bbva.gl.cs.360.gdh.inq.confs.kfcm"

In [3]:
job_hammurabi = {
    "_id":"kfcm-ar-hmm-qlt-custfdbkdetailmaster-01",
    "kind":"processing",
    "description":"hamm-table-custfdbkdetailmaster",
    "runtime":"hammurabi-lts",
    "streaming":False,
    "size":"S",
    "params":{
        "configUrl":"https://artifactory-gdt.central-02.nextgen.igrupobbva/artifactory/gl-datio-spark-libs-maven-local/com/bbva/gl/cs/360/confs/kfcm/2.0.1/hammurabi/confs/master/kfcm/t_kfcm_cust_fdbk_detail-qlm-ri3j_mvpi.conf",
        "sparkHistoryEnabled":"false"
    },
    "env":{
        "XD_ENABLED":"True",
        "myschemacountry":'xx',
        'myschemaversionmaster':'latest'
    },
    "tags":[
        "modeloglobal",
        "gdh",
        "c360",
        "master",
        "hammurabi"
    ]
}
job_kirby = {
    "_id":"kfcm-ar-krb-inm-custfdbk-01",
    "kind":"processing",
    "description":"load-table-custfdbk",
    "runtime":"kirby3-lts",
    "streaming":False,
    "size":"S",
    "params":{
        "configUrl":"https://artifactory-gdt.central-02.nextgen.igrupobbva/artifactory/gl-datio-spark-libs-maven-local/com/bbva/gl/cs/360/confs/kfcm/2.0.1/kirby/confs/master/kfcm/t_kfcm_cust_fdbk-inm-raw_ri3j.conf",
        "sparkHistoryEnabled":"false"
    },
    "env":{
        "XD_ENABLED":"True",
        "myschemacountry":"xx",
        "myschemaversionmaster":"latest",
        "c360_compact":"true",
        "mydbcatalog":"xx_master"
    },
    "tags":[
        "modeloglobal",
        "gdh",
        "c360",
        "master",
        "kirby"
    ]
}
job_smartcleaner = {
    "_id":"ri3j-ar-dfs-rmv-opinatorraw-01",
    "kind":"processing",
    "description":"clean-table-opinatorraw",
    "runtime":"spark:3.1",
    "streaming":False,
    "size":"S",
    "params":{
        "artifactUrl":"https://artifactory-gdt.central-01.nextgen.igrupobbva/artifactory/gl-datio-spark-libs-maven-local/com/bbva/projectsdatio/smartcleaner/0.4.3/smartcleaner-0.4.3-jar-with-dependencies.jar",
        "configUrl":"https://artifactory-gdt.central-01.nextgen.igrupobbva/artifactory/gl-datio-spark-libs-maven-local/com/bbva/c360_datio_generic_config/0.3.0/com/datiodb/smartcleaner/borrado_stg.conf",
        "mainClass":"com.bbva.projectsdatio.pocsmodeloseguridad.smartcleaner.SmartCleaner",
        "sparkHistoryEnabled":"false"
    },
    "env":{
        "time_amount":"7",
        "input_path":"/in/staging/datax/ri3j/t_ri3j_opinator/",
        "frequency":"D",
        "is_directory":"true"
    },
    "tags":[
        "modeloglobal",
        "gdh",
        "c360",
        "smartcleaner",
        "stg"
    ]
}

In [4]:
#Reseteamos directorios (eliminamos y volvemos a crear)
if os.path.exists('./jobs_ingesta/'):
    shutil.rmtree('./jobs_ingesta/')
for p in paises_dict.keys():
    os.makedirs('./jobs_ingesta/'+p+'/hammurabi/')
    os.makedirs('./jobs_ingesta/'+p+'/kirby/')
    if smartcleaner:
        os.makedirs('./jobs_ingesta/'+p+'/smartcleaner/')

In [5]:
kirbys[0].split('-')[0]

't_kfcm_fdbk_filled_form_field'

In [6]:
package = package.replace('.','/')
print('HAMMURABIS')
for i in hammurabis:
    print('-',i)
    for p in paises_dict.keys():
        nombre_job = i[:6][2:] + '-' + 'gl' + '-' + 'hmm' + '-' + 'qlt' + '-' + i[6:].split('-')[0].replace('_','')[:26] + '-' + '01'
        print(nombre_job)
        print('longitud nombre job:(<=46)',len(nombre_job))
        if len(nombre_job) > 46:
             print('NÚMERO DE CARACTERES EXCESIVO, RECORTAR NOMBRE')
             break
        job_hammurabi['description']            = nombre_job
        job_hammurabi['_id']                    = nombre_job
        # job_hammurabi['env']['myschemacountry'] = p
        job_hammurabi['params']['configUrl']    = 'https://artifactory-gdt.central-{0}.nextgen.igrupobbva/artifactory/gl-datio-spark-libs-maven-local/{1}/{2}/hammurabi/confs/master/{3}/{4}'.format(
            paises_dict[p],
            package,
            version,
            i[:6][2:],
            i
            )
        with open('./jobs_ingesta/' + p + '/hammurabi/' + nombre_job + '.json', 'w', encoding='utf-8') as f:
                json.dump(job_hammurabi, f, ensure_ascii=False, indent=4)
print('\nKIRBYS')
for i in kirbys:
    print('-',i)
    for p in paises_dict.keys():
        nombre_job = i[:6][2:] + '-' + 'gl' + '-' + 'krb' + '-' + 'inm' + '-' + i[6:].split('-')[0].replace('_','')[:26] + '-' + '01'
        print(nombre_job)
        print('longitud nombre job:(<=46)',len(nombre_job))
        if len(nombre_job) > 46:
             print('NÚMERO DE CARACTERES EXCESIVO, RECORTAR NOMBRE')
             break
        job_kirby['description']            = nombre_job
        job_kirby['_id']                    = nombre_job
        # job_kirby['env']['myschemacountry'] = p
        # job_kirby['env']['mydbcatalog']     = p + "_master"
        job_kirby['params']['configUrl']    = 'https://artifactory-gdt.central-{0}.nextgen.igrupobbva/artifactory/gl-datio-spark-libs-maven-local/{1}/{2}/kirby/confs/master/{3}/{4}'.format(
            paises_dict[p],
            package,
            version,
            i[:6][2:],
            i
            )
        with open('./jobs_ingesta/' + p + '/kirby/' + nombre_job + '.json', 'w', encoding='utf-8') as f:
            json.dump(job_kirby, f, ensure_ascii=False, indent=4)

if smartcleaner:
    print("\nsmartcleaner")
    for i in kirbys:
        print('-',i)
        for p in paises_dict.keys():
            nombre_job = i[:6][2:] + '-' + 'gl' + '-' + 'dfs' + '-' + 'rmv' + '-' + i[6:].split('-')[0].replace('_','')[:23] + smartcleaner_kind + '-' + '01'
            print(nombre_job)
            print('longitud nombre job:(<=46)',len(nombre_job))
            if len(nombre_job) > 46:
                print('NÚMERO DE CARACTERES EXCESIVO, RECORTAR NOMBRE')
                break
            job_smartcleaner['description']            = nombre_job
            job_smartcleaner['_id']                    = nombre_job

            job_smartcleaner['params']['artifactUrl']  = 'https://artifactory-gdt.central-{0}.nextgen.igrupobbva/artifactory/gl-datio-spark-libs-maven-local/com/bbva/projectsdatio/smartcleaner/0.4.3/smartcleaner-0.4.3-jar-with-dependencies.jar'.format(
                paises_dict[p]
                )
            job_smartcleaner['params']['configUrl']    = 'https://artifactory-gdt.central-{0}.nextgen.igrupobbva/artifactory/gl-datio-spark-libs-maven-local/com/bbva/c360_datio_generic_config/0.3.0/com/datiodb/smartcleaner/borrado_{1}.conf'.format(
                paises_dict[p],
                smartcleaner_kind
                )
            job_smartcleaner['env']['input_path']      = "/in/staging/datax/{0}/{1}/".format(
                i[:6][2:],
                i.split('-')[0]
                )
            with open('./jobs_ingesta/' + p + '/smartcleaner/' + nombre_job + '.json', 'w', encoding='utf-8') as f:
                json.dump(job_smartcleaner, f, ensure_ascii=False, indent=4)


HAMMURABIS
- t_kfcm_fdbk_filled_form_field-qlm-kfcm_ext_mvpi.conf
kfcm-gl-hmm-qlt-fdbkfilledformfield-01
longitud nombre job:(<=46) 38
kfcm-gl-hmm-qlt-fdbkfilledformfield-01
longitud nombre job:(<=46) 38
- t_kfcm_fdbk_saturation_rules-qlm-kfcm_ext_mvpi.conf
kfcm-gl-hmm-qlt-fdbksaturationrules-01
longitud nombre job:(<=46) 38
kfcm-gl-hmm-qlt-fdbksaturationrules-01
longitud nombre job:(<=46) 38
- t_kfcm_feedback_filled_form-qlm-kfcm_ext_mvpi.conf
kfcm-gl-hmm-qlt-feedbackfilledform-01
longitud nombre job:(<=46) 37
kfcm-gl-hmm-qlt-feedbackfilledform-01
longitud nombre job:(<=46) 37
- t_kfcm_feedback_form_field-qlm-kfcm_ext_mvpi.conf
kfcm-gl-hmm-qlt-feedbackformfield-01
longitud nombre job:(<=46) 36
kfcm-gl-hmm-qlt-feedbackformfield-01
longitud nombre job:(<=46) 36
- t_kfcm_feedback_form_shown-qlm-kfcm_ext_mvpi.conf
kfcm-gl-hmm-qlt-feedbackformshown-01
longitud nombre job:(<=46) 36
kfcm-gl-hmm-qlt-feedbackformshown-01
longitud nombre job:(<=46) 36
- t_kfcm_feedback_forms-qlm-kfcm_ext_mvpi.c

In [7]:
job_hammurabi

{'_id': 'kfcm-gl-hmm-qlt-feedbackoptions-01',
 'kind': 'processing',
 'description': 'kfcm-gl-hmm-qlt-feedbackoptions-01',
 'runtime': 'hammurabi-lts',
 'streaming': False,
 'size': 'S',
 'params': {'configUrl': 'https://artifactory-gdt.central-02.nextgen.igrupobbva/artifactory/gl-datio-spark-libs-maven-local/com/bbva/gl/cs/360/gdh/inq/confs/kfcm/2.1.2/hammurabi/confs/master/kfcm/t_kfcm_feedback_options-qlm-kfcm_ext_mvpi.conf',
  'sparkHistoryEnabled': 'false'},
 'env': {'XD_ENABLED': 'True',
  'myschemacountry': 'xx',
  'myschemaversionmaster': 'latest'},
 'tags': ['modeloglobal', 'gdh', 'c360', 'master', 'hammurabi']}

In [8]:
job_kirby

{'_id': 'kfcm-gl-krb-inm-feedbackoptions-01',
 'kind': 'processing',
 'description': 'kfcm-gl-krb-inm-feedbackoptions-01',
 'runtime': 'kirby3-lts',
 'streaming': False,
 'size': 'S',
 'params': {'configUrl': 'https://artifactory-gdt.central-02.nextgen.igrupobbva/artifactory/gl-datio-spark-libs-maven-local/com/bbva/gl/cs/360/gdh/inq/confs/kfcm/2.1.2/kirby/confs/master/kfcm/t_kfcm_feedback_options-inm-stg_kfcm_ext.conf',
  'sparkHistoryEnabled': 'false'},
 'env': {'XD_ENABLED': 'True',
  'myschemacountry': 'xx',
  'myschemaversionmaster': 'latest',
  'c360_compact': 'true',
  'mydbcatalog': 'xx_master'},
 'tags': ['modeloglobal', 'gdh', 'c360', 'master', 'kirby']}

In [9]:
job_smartcleaner

{'_id': 'kfcm-gl-dfs-rmv-feedbackoptionsstg-01',
 'kind': 'processing',
 'description': 'kfcm-gl-dfs-rmv-feedbackoptionsstg-01',
 'runtime': 'spark:3.1',
 'streaming': False,
 'size': 'S',
 'params': {'artifactUrl': 'https://artifactory-gdt.central-02.nextgen.igrupobbva/artifactory/gl-datio-spark-libs-maven-local/com/bbva/projectsdatio/smartcleaner/0.4.3/smartcleaner-0.4.3-jar-with-dependencies.jar',
  'configUrl': 'https://artifactory-gdt.central-02.nextgen.igrupobbva/artifactory/gl-datio-spark-libs-maven-local/com/bbva/c360_datio_generic_config/0.3.0/com/datiodb/smartcleaner/borrado_stg.conf',
  'mainClass': 'com.bbva.projectsdatio.pocsmodeloseguridad.smartcleaner.SmartCleaner',
  'sparkHistoryEnabled': 'false'},
 'env': {'time_amount': '7',
  'input_path': '/in/staging/datax/kfcm/t_kfcm_feedback_options/',
  'frequency': 'D',
  'is_directory': 'true'},
 'tags': ['modeloglobal', 'gdh', 'c360', 'smartcleaner', 'stg']}